## Purpose

This document is intended for professor to test individual intention implementations. Each test ensures that specific intents behave as expected when given predefined inputs.

## Examples of Messages to Test Each Intention

1. **Intent Name**  
   _e.g., Retrieve Company Info_

2. **Example Inputs**  
   - "Tell me about the company."  
   - "What is your mission statement?"

3. **Expected Outputs**  
   - "Our company specializes in renewable energy."  
   - "The mission is to provide eco-friendly energy solutions."

In [2]:
# Now you can try importing your module
from cobuy.chatbot.chains.product_info import ProductInfoReasoningChain, ProductInfoResponseChain